In [ ]:
import pandas as pd
import nltk
import numpy as np
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from itertools import repeat
import networkx as nx
from gensim.models import Word2Vec
from tqdm import tqdm

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# all_data = pd.read_csv('./tcc_ceds_music.csv') #the dataset that you want to obtain a graph
all_data = pd.read_csv("https://gist.githubusercontent.com/ivangermanov/f882d48647cc7457d2cb99e0f27eb8b7/raw/9909e23b85bb146e78da1d7c4b71ea20b2262d55/processed.csv")

In [ ]:
all_data = all_data.sample(frac=1).reset_index(drop=True)
all_data

,Unnamed: 0,artist_name,track_name,release_date,genre,lyrics,len,dating,violence,world/life,night/time,shake the audience,family/gospel,romantic,communication,obscene,music,movement/places,light/visual perceptions,family/spiritual,like/girls,sadness,feelings,danceability,loudness,acousticness,instrumentalness,valence,energy,topic,age,lang,processed_lyrics
0,27913,kenny rogers,the night goes on,1984,country,walk land lose stare time time cause lonely ma...,111,0.000478,0.000478,0.000478,0.247371,0.000478,0.000478,0.000478,0.038899,0.000478,0.000478,0.000478,0.000478,0.049877,0.000478,0.465991,0.180312,0.724900,0.525242,0.604417,0.000000,0.759893,0.372353,sadness,0.514286,en,afraid alright apart away belong caus danger e...
1,42114,b.b. king,night life,1967,blues,go yeah hangin night life good life life yeah ...,39,0.001422,0.001422,0.316640,0.077187,0.116534,0.001422,0.001422,0.001422,0.169154,0.198151,0.001422,0.001422,0.001422,0.001422,0.001422,0.103842,0.248348,0.623644,0.392570,0.030466,0.590890,0.473457,world/life,0.757143,en,avenu blue break dream dreamin go good hangin ...
2,1104,chubby checker,the jet,1961,pop,survive jett blackhearts roll cure like heaven...,22,0.006579,0.341851,0.006579,0.006579,0.006579,0.006579,0.006579,0.006579,0.006579,0.006579,0.006579,0.006579,0.309184,0.109103,0.006579,0.006579,0.585184,0.781812,0.474899,0.237854,0.974237,0.699690,violence,0.842857,en,american begin blackheart bowi carpent checker...
3,41164,b.b. king,don't touch,1962,blues,ready guy wooh okay naive beaters keepin cool ...,44,0.002193,0.002193,0.002193,0.002193,0.002193,0.002193,0.409809,0.002193,0.456034,0.002193,0.002193,0.002193,0.002193,0.002193,0.002193,0.002193,0.849453,0.712150,0.810241,0.215587,0.945383,0.626615,obscene,0.828571,en,achiev attract beater caus clap cool creat dok...
4,67782,passafire,unfamiliar,2007,reggae,want bigger mind waste time want throw ridge l...,41,0.001754,0.001754,0.170462,0.459576,0.001754,0.001754,0.038152,0.001754,0.001754,0.038971,0.001754,0.001754,0.001754,0.001754,0.161282,0.001754,0.809379,0.744532,0.118473,0.000727,0.301319,0.356336,night/time,0.185714,en,bewild bigger care dare gear goodby guitar lea...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17772,6595,elvis costello,watching the detectives,1977,pop,nice girls defect cellophane shrinkwrapped cor...,125,0.019686,0.240627,0.000711,0.025155,0.000711,0.016176,0.000711,0.000711,0.360462,0.000711,0.000711,0.000711,0.000711,0.054342,0.275017,0.000711,0.788801,0.600620,0.015662,0.000013,0.815540,0.366347,obscene,0.614286,en,angri away babi beat beg belong blow caus cell...
17773,8486,the the,soul mining,1983,pop,float tunnel little wooden cold lonely envelop...,149,0.000521,0.000521,0.000521,0.157344,0.000521,0.010297,0.000521,0.162064,0.000521,0.000521,0.000521,0.035131,0.000521,0.070704,0.408489,0.149718,0.395646,0.462144,0.683735,0.025304,0.098310,0.314293,sadness,0.528571,en,book captur caus circl cold depth drift envelo...
17774,1574,bobby vinton,long lonely nights,1962,pop,long long lonely nights eye wond ring right le...,62,0.039919,0.000907,0.000907,0.327993,0.035607,0.000907,0.000907,0.000907,0.000907,0.147037,0.000907,0.000907,0.000907,0.000907,0.382390,0.055258,0.252681,0.685768,0.839357,0.000000,0.251855,0.365346,sadness,0.828571,en,break come dear doorway eye face go guess hear...
17775,25916,johnny cash,it'll be her,1978,country,find better love go care love woman know roll ...,21,0.002506,0.002506,0.002506,0.002506,0.061598,0.002506,0.658236,0.147486,0.002506,0.002506,0.002506,0.002506,0.002506,0.002506,0.002506,0.046162,0.581934,0.545086,0.666666,0.000005,0.408491,0.291269,romantic,0.600000,en,better care cold find go hold know love need r...


In [ ]:
#Obtain Node Dataframe Method
def obtainNodes(musicData): 
    wordNodes = obtainWordNodes(musicData)
    song_names = musicData['track_name'].tolist()
    genres = musicData['genre'].tolist()
    no_of_songs = len(musicData['artist_name'])
    no_of_words = len(wordNodes)
    type_list=[]
    type_list.extend(repeat('song',no_of_songs))
    type_list.extend(repeat('word',no_of_words))
    wordGenres = wordNodes['genre'].tolist()
    genres += wordGenres
    node_ids = [*range(0, no_of_words + no_of_songs)]
    node_names = [y for x in [song_names, wordNodes['word'].tolist()] for y in x]
    list_of_tuples = list(zip(node_ids, node_names, type_list, genres))
    df = pd.DataFrame(list_of_tuples,
                  columns = ['node_id', 'node_name', 'node_type', 'genre'])
    return musicData, df, wordNodes
    
def obtainWordNodes(processedMusicData):
    unique_words = {}
    for i in range(len(processedMusicData['processed_lyrics'])):
        words = processedMusicData['processed_lyrics'][i].split(' ')
        for word in words:
            unique_words[word] = processedMusicData['genre'][i]

    keyList = list(unique_words.keys())
    
    valueList=list(unique_words.values())
    node_ids = [*range(0, len(unique_words))]
    list_of_tuples = list(zip(node_ids, keyList, valueList))
    df = pd.DataFrame(list_of_tuples,
                  columns = ['node_id', 'word', 'genre'])
    return df

def obtainUniqueWords(musicData):
    print(musicData['processed_lyrics'])
    unique_words = []
    res_df = {}
    for sentence in musicData['processed_lyrics']:
        words = sentence.split(' ')
        unique_words.append(words)
    return unique_words

vector_size = 100
unique_words = obtainUniqueWords(all_data)
model = Word2Vec(sentences=unique_words, size=vector_size, window=5, min_count=1, workers=4)

def obtainVectors(musicData, wordData):
  res_df = []
  word_df = []
  for row in tqdm(range(len(musicData))):
        words = musicData.iloc[row]['processed_lyrics'].split(' ')
        empty_list = [0.0 for i in range(0,vector_size)]
        vec = np.array(empty_list)
        noOfCount = 0
        for word in words:
            vec = np.add(vec, model.wv[word])
            noOfCount +=1
        vec = np.divide(vec, noOfCount)
        res_df.append( vec)
  for row in tqdm(range(len(wordData))):
    check = wordData['word'][row].split(' ')
    if(len(check) == 1 ):
      word_df.append(model.wv[wordData['word'][row]])

  return res_df, word_df

0        afraid alright apart away belong caus danger e...
1        avenu blue break dream dreamin go good hangin ...
2        american begin blackheart bowi carpent checker...
3        achiev attract beater caus clap cool creat dok...
4        bewild bigger care dare gear goodby guitar lea...
                               ...                        
17772    angri away babi beat beg belong blow caus cell...
17773    book captur caus circl cold depth drift envelo...
17774    break come dear doorway eye face go guess hear...
17775    better care cold find go hold know love need r...
17776    add bang bell belong board bone break build ca...
Name: processed_lyrics, Length: 17777, dtype: object


In [ ]:
processed_music, nodes, words = obtainNodes(all_data)
song_vectors, word_vectors = obtainVectors(all_data, words)
empty_list = [0.0 for i in range(0,100)]
vec = np.array(empty_list)

G = nx.Graph()
wordCounter = 0
for i in tqdm(range(len(nodes))):
    if i < len(song_vectors):
        G.add_node(i, 
               node_id = str(nodes.iloc[i]['node_id']), 
               node_name = str(nodes.iloc[i]['node_name']), 
               node_type = str(nodes.iloc[i]['node_type']), 
               genre = str(nodes.iloc[i]['genre']),
               vector = str(song_vectors[i]))
    else:
      check = nodes.iloc[i]['node_name'].split(' ')
      if (len(check) == 1):
        G.add_node(i, 
               node_id = str(nodes.iloc[i]['node_id']), 
               node_name = str(nodes.iloc[i]['node_name']), 
               node_type = str(nodes.iloc[i]['node_type']), 
               genre = str(nodes.iloc[i]['genre']),
               vector = str(word_vectors[wordCounter]))
        wordCounter += 1
      else:
            vec = np.array(empty_list)
            G.add_node(i, 
               node_id = str(nodes.iloc[i]['node_id']), 
               node_name = str(nodes.iloc[i]['node_name']), 
               node_type = str(nodes.iloc[i]['node_type']), 
               genre = str(nodes.iloc[i]['genre']),
               vector = str(vec))

['country', 'blues', 'pop', 'blues', 'reggae', 'country', 'pop', 'country', 'rock', 'reggae', 'rock', 'pop', 'rock', 'blues', 'pop', 'jazz', 'jazz', 'blues', 'pop', 'pop', 'reggae', 'rock', 'hip hop', 'country', 'pop', 'pop', 'rock', 'country', 'rock', 'blues', 'rock', 'rock', 'jazz', 'reggae', 'jazz', 'pop', 'pop', 'jazz', 'pop', 'rock', 'pop', 'pop', 'pop', 'country', 'pop', 'reggae', 'rock', 'country', 'hip hop', 'reggae', 'pop', 'jazz', 'jazz', 'country', 'rock', 'pop', 'blues', 'country', 'blues', 'pop', 'country', 'pop', 'rock', 'rock', 'jazz', 'blues', 'pop', 'country', 'pop', 'country', 'country', 'pop', 'country', 'blues', 'rock', 'blues', 'pop', 'rock', 'blues', 'blues', 'reggae', 'jazz', 'blues', 'pop', 'blues', 'jazz', 'country', 'pop', 'blues', 'pop', 'reggae', 'pop', 'reggae', 'pop', 'rock', 'country', 'jazz', 'hip hop', 'jazz', 'jazz', 'reggae', 'country', 'rock', 'blues', 'blues', 'country', 'pop', 'jazz', 'pop', 'country', 'jazz', 'blues', 'pop', 'reggae', 'blues', 'ja

100%|██████████| 51139/51139 [01:40<00:00, 507.63it/s]

51139


In [ ]:
#Construct edges Method
def get_node_indexes(song, nodes):
    list_words = []
    words = song.split(' ')
    for word in words:    
        list_words.append(int(nodes['node_id'][nodes['node_name'] == word].iloc[-1]))
    return list_words
def get_artist_index(song_artist, nodes):
    return nodes['node_id'][nodes['node_name'] == song_artist]
    
for i in tqdm(range(len(processed_music))):
    node_indexes = get_node_indexes(processed_music['processed_lyrics'][i],nodes)
    for j in node_indexes:
        G.add_edge(i, j)
    artist_index = get_artist_index(processed_music['artist_name'][i],nodes)
    for j in artist_index:
        G.add_edge(i, j)

100%|██████████| 17777/17777 [58:24<00:00,  5.07it/s]

744314


In [ ]:
nx.write_gml(G, "graph_gcn.gml")